## Generate spectrograms for all downloaded tracks

Find all tracks, then use ffmpeg to convert them to wav. Then, using matplotlib functions, create grayscale spectrograms and save them as .png files.

**This notebook has some issues, jupyter doesn't seem to want to play nicely with subprocess. Use generatespec.py instead.**

In [3]:
import glob
from multiprocessing import Pool
from tqdm import tnrange, tqdm_notebook
import os
from tempfile import mktemp
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import subprocess
import ffmpeg

dpi = 92
resolution = 128
ffmpeg_opts = ['-ac', '1']

def make_spectrogram(filename):
    wname = mktemp('.wav')
    #ffmpeg_call = ['ffmpeg', '-i', filename, *ffmpeg_opts, wname]
    #subprocess.check_call(ffmpeg_call)
    ffmpeg.input(filename).output(wname, ac=1).run()
    sample_rate, samples = wavfile.read(wname)
    os.unlink(wname)

    fig, ax = plt.subplots(frameon=False)
    fig.set_size_inches(resolution / dpi, resolution / dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.specgram(samples, Fs=sample_rate, cmap='Greys_r')
    fname = os.path.abspath(filename.replace('.mp3', '.png'))
    fig.savefig(fname, dpi=92)
    plt.close()


files = glob.glob('tracks/**/*.mp3', recursive=True) + glob.glob('tracks/**/*.m4a', recursive=True)

p = Pool(15)
r = p.imap(make_spectrogram, files)
# Workaround to make the progress bar work, for some reason it doesnt work properly with subprocess
for i in tqdm_notebook(r, total=len(files)):
    pass



                                        types |   # objects |   total size
============================================= | =========== | ============
                                 <class 'dict |         142 |     45.46 KB
                    <class 'collections.deque |          11 |      6.79 KB
                                 <class 'list |          38 |      5.12 KB
                              <class 'weakref |          39 |      3.05 KB
                                  <class 'set |           0 |      3.00 KB
                                  <class 'str |          37 |      2.73 KB
       <class 'traitlets.config.loader.Config |           8 |      2.00 KB
           <class 'builtin_function_or_method |          27 |      1.90 KB
                                <class 'bytes |          21 |      1.38 KB
                                  <class 'int |          46 |      1.29 KB
                               <class 'method |          19 |      1.19 KB
                        

In [3]:
display(files)

['tracks/d/d/d/5/ddd553d4-977e-416c-8f57-e4b72c0fc746-7ae8aa6d.mp3',
 'tracks/d/d/d/5/ddd553d4-977e-416c-8f57-e4b72c0fc746-132141b5.mp3',
 'tracks/d/d/d/5/ddd553d4-977e-416c-8f57-e4b72c0fc746-dad77823.mp3',
 'tracks/d/d/d/4/ddd4c4bc-a4ae-4946-b59c-a86aea7fa4f0-cee7e885.mp3',
 'tracks/d/d/d/4/ddd4c4bc-a4ae-4946-b59c-a86aea7fa4f0-86c12fd1.mp3',
 'tracks/d/d/d/4/ddd4c4bc-a4ae-4946-b59c-a86aea7fa4f0-ceb7c1b2.mp3',
 'tracks/d/d/9/9/dd999ced-1dd4-44b8-ab67-ee5ec23e52b7-6100f2f6.mp3',
 'tracks/d/d/9/9/dd999ced-1dd4-44b8-ab67-ee5ec23e52b7-f06f8368.mp3',
 'tracks/d/d/9/9/dd999ced-1dd4-44b8-ab67-ee5ec23e52b7-2ca322fa.mp3',
 'tracks/d/d/9/1/dd91eb4b-6d08-4c14-9eb2-197a02a35f6c-4e45f115.mp3']

KeyboardInterrupt: 